In [22]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, Subset, ConcatDataset
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
import time
from tqdm import tqdm
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
MEAN, STD = [0.4823]*3, [0.2216]*3
BATCH_SIZE = 64

In [19]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'chest-xray-pneumonia' dataset.
Path to dataset files: /kaggle/input/chest-xray-pneumonia


In [20]:
train_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

val_test_tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

base_path = os.path.join(path, "chest_xray")
full_train = datasets.ImageFolder(os.path.join(base_path, "train"), transform=train_tf)
train_clean = datasets.ImageFolder(os.path.join(base_path, "train"), transform=val_test_tf)

indices = torch.randperm(len(full_train), generator=torch.Generator().manual_seed(42))
train_idx, val_idx = indices[:int(0.9*len(full_train))], indices[int(0.9*len(full_train)):]

train_loader = DataLoader(Subset(full_train, train_idx), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(ConcatDataset([datasets.ImageFolder(os.path.join(base_path, "val"), transform=val_test_tf), 
                                       Subset(train_clean, val_idx)]), batch_size=BATCH_SIZE)

# Architecture baseline implémenté

In [21]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        
        # Bloc 1 : 224x224 -> 112x112
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        
        # Bloc 2 : 112x112 -> 56x56
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        # Bloc 3 : 56x56 -> 28x28
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        # Bloc 4 (Ajouté pour le budget paramètres) : 28x28 -> 14x14
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # MLP Final : Entrée 64 * 14 * 14 = 12 544
        self.fc1 = nn.Linear(64 * 14 * 14, 128) # Réduit à 128 pour l'efficacité
        self.fc2 = nn.Linear(128, num_classes)
        
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x)))) # 4ème réduction
        
        x = x.view(x.size(0), -1) # Flatten propre
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [23]:
model = SimpleCNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

weights = torch.tensor([2.8, 1.0]).to(device) 
criterion = nn.CrossEntropyLoss(weight=weights)

writer = SummaryWriter('runs/pneumonia_baseline_1')

In [24]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Nombre de paramètres : {total_params:,}")
# On devrait être autour de 1.2M - 1.5M selon les tailles des FC.

Nombre de paramètres : 1,666,882


# Trainning loop

In [25]:
# Test rapide : Est-ce que les loaders existent ?
try:
    print(f"Train loader : {type(train_loader)} - OK")
    print(f"Val loader : {type(val_loader)} - OK")
    # On vérifie si on peut tirer un batch
    images, labels = next(iter(train_loader))
    print(f"Batch testé : {images.shape}")
except NameError as e:
    print(f"❌ Erreur : La variable n'existe pas. {e}")
except Exception as e:
    print(f"❌ Autre erreur : {e}")

Train loader : <class 'torch.utils.data.dataloader.DataLoader'> - OK
Val loader : <class 'torch.utils.data.dataloader.DataLoader'> - OK
Batch testé : torch.Size([64, 3, 224, 224])


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        print(f"\nÉpoque {epoch+1}/{epochs}")
        for images, labels in tqdm(train_loader, desc="Training"):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = 100 * correct_train / total_train
        
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                running_val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
        
        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        epoch_val_acc = 100 * correct_val / total_val
        
        # Logging tensorboard
        writer.add_scalar('Loss/Train', epoch_train_loss, epoch)
        writer.add_scalar('Loss/Val', epoch_val_loss, epoch)
        writer.add_scalar('Accuracy/Train', epoch_train_acc, epoch)
        writer.add_scalar('Accuracy/Val', epoch_val_acc, epoch)
        
        print(f"Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.2f}%")
        print(f"Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.2f}%")
        
        # Sauvegarde du meilleur modèle
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"⭐ Nouveau meilleur modèle sauvegardé ! (Acc: {best_val_acc:.2f}%)")

    writer.close()
    print("\nEntraînement terminé !")

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)


Époque 1/10


Training: 100%|██████████| 74/74 [06:43<00:00,  5.45s/it]


Train Loss: 0.4882 | Train Acc: 86.96%
Val Loss: 0.1019 | Val Acc: 96.28%
⭐ Nouveau meilleur modèle sauvegardé ! (Acc: 96.28%)

Époque 2/10


Training: 100%|██████████| 74/74 [06:44<00:00,  5.46s/it]


Train Loss: 0.1560 | Train Acc: 93.63%
Val Loss: 0.1478 | Val Acc: 96.84%
⭐ Nouveau meilleur modèle sauvegardé ! (Acc: 96.84%)

Époque 3/10


...

Training: 100%|██████████| 74/74 [06:44<00:00,  5.47s/it]


Train Loss: 0.0992 | Train Acc: 96.38%
Val Loss: 0.0782 | Val Acc: 97.77%
⭐ Nouveau meilleur modèle sauvegardé ! (Acc: 97.77%)

Époque 9/10


Training: 100%|██████████| 74/74 [06:44<00:00,  5.47s/it]


Train Loss: 0.0844 | Train Acc: 96.51%
Val Loss: 0.1017 | Val Acc: 96.84%

Époque 10/10


Training:   4%|▍         | 3/74 [00:16<06:27,  5.46s/it]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs